In [ ]:
# Hệ Thống Gợi Ý Món Ăn Việt Nam với Word2Vec

Xây dựng hệ thống gợi ý món ăn dựa trên các đặc trưng:
- Type (Loại món)
- Name (Tên món)
- Score-text (Điểm đánh giá)
- Main Ingredients (Nguyên liệu chính)
- Calories (Calo)
- Suitable Time (Thời gian phù hợp)

## Bước 1: Import thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings
warnings.filterwarnings('ignore')

## Bước 2: Đọc dữ liệu

In [2]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv('train.csv')

# Hiển thị thông tin cơ bản
print(f"Số lượng món ăn: {len(df)}")
print(f"\nCác cột trong dataset:")
print(df.columns.tolist())
print(f"\nDữ liệu mẫu:")
df.head()

Số lượng món ăn: 199

Các cột trong dataset:
['STT', 'type', 'name', 'score-text', 'Main Ingredients', 'Calories (Approx.)', 'Suitable Time', 'Origin / Province']

Dữ liệu mẫu:


,STT,type,name,score-text,Main Ingredients,Calories (Approx.),Suitable Time,Origin / Province
0,1,Noodle Soup,Pho,4.4,"Rice noodles, bone broth (beef/chicken), herbs...",350 - 500,Breakfast / Dinner,Nam Dinh / Hanoi (Northern Vietnam)
1,2,Sandwich type,Bánh mì,4.6,"Baguette, pate, cucumber, cilantro, pickled ca...",350 - 500,Breakfast / Lunch / Snack,Nationwide (Popularized in Saigon)
2,3,Pancake,Crispy Vietnamese Pancake (Bánh xèo),4.2,"Rice flour, turmeric, coconut milk, shrimp, po...",300 - 450 (per cake),Lunch / Dinner,South & Central Vietnam
3,4,Appetizer,Vietnamese Fried Spring Rolls (Chả giò),4.3,"Minced pork, mushroom, glass noodles, egg, ric...",150 - 200 (per roll),Appetizer / Main Course,Nationwide
4,5,Side Dish,Vietnamese Crullers (Quẩy),3.7,"Wheat flour, baking powder, yeast (deep-fried)",100 - 150 (per pair),Breakfast (Side dish for Pho/Porridge),Nationwide (Chinese origin)


## Bước 3: Tiền xử lý dữ liệu

In [3]:
# Kiểm tra dữ liệu thiếu
print("Kiểm tra dữ liệu thiếu:")
print(df.isnull().sum())
print("\n" + "="*50)

# Xử lý dữ liệu thiếu (nếu có)
df = df.fillna('')

# Hàm tiền xử lý text
def preprocess_text(text):
    """Chuyển text thành chữ thường và loại bỏ ký tự đặc biệt"""
    if pd.isna(text) or text == '':
        return ''
    text = str(text).lower()
    # Giữ lại dấu phẩy, dấu gạch ngang và khoảng trắng
    text = re.sub(r'[^a-z0-9\s,/-]', '', text)
    return text

# Tiền xử lý các cột
df['type_processed'] = df['type'].apply(preprocess_text)
df['name_processed'] = df['name'].apply(preprocess_text)
df['score_processed'] = df['score-text'].apply(lambda x: str(x).replace('.', ''))
df['ingredients_processed'] = df['Main Ingredients'].apply(preprocess_text)
df['calories_processed'] = df['Calories (Approx.)'].apply(preprocess_text)
df['time_processed'] = df['Suitable Time'].apply(preprocess_text)

print("\nDữ liệu sau khi tiền xử lý:")
df[['name', 'type_processed', 'ingredients_processed', 'time_processed']].head()

Kiểm tra dữ liệu thiếu:
STT                    0
type                   0
name                   0
score-text            81
Main Ingredients       0
Calories (Approx.)     0
Suitable Time          0
Origin / Province      0
dtype: int64


Dữ liệu sau khi tiền xử lý:


,name,type_processed,ingredients_processed,time_processed
0,Pho,noodle soup,"rice noodles, bone broth beef/chicken, herbs, ...",breakfast / dinner
1,Bánh mì,sandwich type,"baguette, pate, cucumber, cilantro, pickled ca...",breakfast / lunch / snack
2,Crispy Vietnamese Pancake (Bánh xèo),pancake,"rice flour, turmeric, coconut milk, shrimp, po...",lunch / dinner
3,Vietnamese Fried Spring Rolls (Chả giò),appetizer,"minced pork, mushroom, glass noodles, egg, ric...",appetizer / main course
4,Vietnamese Crullers (Quẩy),side dish,"wheat flour, baking powder, yeast deep-fried",breakfast side dish for pho/porridge


## Bước 4: Tạo corpus cho Word2Vec

In [4]:
# Tạo câu (sentence) cho mỗi món ăn bằng cách kết hợp các đặc trưng
def create_sentence(row):
    """Tạo câu từ các đặc trưng của món ăn"""
    # Tách các thành phần thành từng từ
    type_words = row['type_processed'].split()
    ingredients_words = row['ingredients_processed'].replace(',', ' ').split()
    calories_words = row['calories_processed'].split()
    time_words = row['time_processed'].replace('/', ' ').split()
    
    # Kết hợp tất cả thành một câu
    sentence = type_words + ingredients_words + calories_words + time_words
    
    # Loại bỏ các từ rỗng
    sentence = [word for word in sentence if word.strip() != '']
    
    return sentence

# Tạo corpus
corpus = []
for idx, row in df.iterrows():
    sentence = create_sentence(row)
    if len(sentence) > 0:
        corpus.append(sentence)

print(f"Số lượng câu trong corpus: {len(corpus)}")
print(f"\nVí dụ 3 câu đầu tiên:")
for i in range(min(3, len(corpus))):
    print(f"{i+1}. {corpus[i]}")

Số lượng câu trong corpus: 199

Ví dụ 3 câu đầu tiên:
1. ['noodle', 'soup', 'rice', 'noodles', 'bone', 'broth', 'beef/chicken', 'herbs', 'onion', '350', '-', '500', 'breakfast', 'dinner']
2. ['sandwich', 'type', 'baguette', 'pate', 'cucumber', 'cilantro', 'pickled', 'carrots', 'chili', '350', '-', '500', 'breakfast', 'lunch', 'snack']
3. ['pancake', 'rice', 'flour', 'turmeric', 'coconut', 'milk', 'shrimp', 'pork', 'bean', 'sprouts', '300', '-', '450', 'per', 'cake', 'lunch', 'dinner']


## Bước 5: Huấn luyện mô hình Word2Vec

In [5]:
# Huấn luyện mô hình Word2Vec
# vector_size: Kích thước vector embedding
# window: Kích thước cửa sổ ngữ cảnh
# min_count: Bỏ qua các từ xuất hiện ít hơn min_count lần
# workers: Số luồng để huấn luyện
# sg: 1 = skip-gram, 0 = CBOW

model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1,  # Skip-gram
    epochs=100
)

print("Mô hình Word2Vec đã được huấn luyện!")
print(f"Số lượng từ trong vocabulary: {len(model.wv)}")
print(f"\nVí dụ một số từ trong vocabulary:")
print(list(model.wv.index_to_key)[:20])

Mô hình Word2Vec đã được huấn luyện!
Số lượng từ trong vocabulary: 527

Ví dụ một số từ trong vocabulary:
['-', 'rice', 'dish', 'breakfast', 'pork', 'snack', 'lunch', '300', '400', 'dinner', 'sauce', '150', '200', 'soup', 'fish', 'per', '500', 'dessert', 'noodle', '350']


## Bước 6: Tạo vector đại diện cho mỗi món ăn

In [7]:
def get_dish_vector(sentence, model):
    """Tính vector trung bình cho một món ăn"""
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word])
    
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Tạo vector cho tất cả các món ăn
dish_vectors = []
for sentence in corpus:
    vector = get_dish_vector(sentence, model)
    dish_vectors.append(vector)

dish_vectors = np.array(dish_vectors)
print(f"Shape của ma trận vector: {dish_vectors.shape}")
print(f"Mỗi món ăn được biểu diễn bằng vector {model.vector_size} chiều")

Shape của ma trận vector: (199, 100)
Mỗi món ăn được biểu diễn bằng vector 100 chiều


In [8]:
# Lưu mô hình Word2Vec
model_path = 'word2vec_food_model.model'
model.save(model_path)
print(f"✅ Mô hình đã được lưu tại: {model_path}")

# Lưu dish_vectors để sử dụng sau
import pickle
vectors_path = 'dish_vectors.pkl'
with open(vectors_path, 'wb') as f:
    pickle.dump(dish_vectors, f)
print(f"✅ Dish vectors đã được lưu tại: {vectors_path}")

# Lưu DataFrame đã xử lý
df_path = 'processed_data.csv'
df.to_csv(df_path, index=False)
print(f"✅ Dữ liệu đã xử lý được lưu tại: {df_path}")

✅ Mô hình đã được lưu tại: word2vec_food_model.model
✅ Dish vectors đã được lưu tại: dish_vectors.pkl
✅ Dữ liệu đã xử lý được lưu tại: processed_data.csv


In [1]:
from gensim.models import Word2Vec
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load mô hình Word2Vec
print("🔄 Đang load mô hình...")
model = Word2Vec.load('word2vec_food_model.model')
print("✅ Đã load mô hình Word2Vec")

# Load dish_vectors
with open('dish_vectors.pkl', 'rb') as f:
    dish_vectors = pickle.load(f)
print("✅ Đã load dish vectors")

# Load DataFrame
df = pd.read_csv('processed_data.csv')
print("✅ Đã load dữ liệu")

print(f"\n📊 Thông tin:")
print(f"  - Số lượng món ăn: {len(df)}")
print(f"  - Kích thước vocabulary: {len(model.wv)}")
print(f"  - Shape dish_vectors: {dish_vectors.shape}")

🔄 Đang load mô hình...
✅ Đã load mô hình Word2Vec
✅ Đã load dish vectors
✅ Đã load dữ liệu

📊 Thông tin:
  - Số lượng món ăn: 199
  - Kích thước vocabulary: 527
  - Shape dish_vectors: (199, 100)


## Bước 7: Xây dựng hàm gợi ý món ăn

In [12]:
df.head()

,STT,type,name,score-text,Main Ingredients,Calories (Approx.),Suitable Time,Origin / Province,type_processed,name_processed,score_processed,ingredients_processed,calories_processed,time_processed
0,1,Noodle Soup,Pho,4.4,"Rice noodles, bone broth (beef/chicken), herbs...",350 - 500,Breakfast / Dinner,Nam Dinh / Hanoi (Northern Vietnam),noodle soup,pho,44.0,"rice noodles, bone broth beef/chicken, herbs, ...",350 - 500,breakfast / dinner
1,2,Sandwich type,Bánh mì,4.6,"Baguette, pate, cucumber, cilantro, pickled ca...",350 - 500,Breakfast / Lunch / Snack,Nationwide (Popularized in Saigon),sandwich type,bnh m,46.0,"baguette, pate, cucumber, cilantro, pickled ca...",350 - 500,breakfast / lunch / snack
2,3,Pancake,Crispy Vietnamese Pancake (Bánh xèo),4.2,"Rice flour, turmeric, coconut milk, shrimp, po...",300 - 450 (per cake),Lunch / Dinner,South & Central Vietnam,pancake,crispy vietnamese pancake bnh xo,42.0,"rice flour, turmeric, coconut milk, shrimp, po...",300 - 450 per cake,lunch / dinner
3,4,Appetizer,Vietnamese Fried Spring Rolls (Chả giò),4.3,"Minced pork, mushroom, glass noodles, egg, ric...",150 - 200 (per roll),Appetizer / Main Course,Nationwide,appetizer,vietnamese fried spring rolls ch gi,43.0,"minced pork, mushroom, glass noodles, egg, ric...",150 - 200 per roll,appetizer / main course
4,5,Side Dish,Vietnamese Crullers (Quẩy),3.7,"Wheat flour, baking powder, yeast (deep-fried)",100 - 150 (per pair),Breakfast (Side dish for Pho/Porridge),Nationwide (Chinese origin),side dish,vietnamese crullers quy,37.0,"wheat flour, baking powder, yeast deep-fried",100 - 150 per pair,breakfast side dish for pho/porridge


In [2]:
def recommend_dishes(dish_name, top_n=5):
    """
    Gợi ý món ăn dựa trên tên món
    
    Parameters:
    - dish_name: Tên món ăn (có thể viết không dấu)
    - top_n: Số lượng món ăn gợi ý
    
    Returns:
    - DataFrame chứa các món ăn được gợi ý
    """
    # Tìm món ăn trong dataset
    dish_name_lower = dish_name.lower()
    matches = df[df['name'].str.lower().str.contains(dish_name_lower, na=False)]
    
    if len(matches) == 0:
        print(f"❌ Không tìm thấy món ăn '{dish_name}'")
        return None
    
    # Lấy index của món ăn đầu tiên tìm được
    dish_idx = matches.index[0]
    print(f"✅ Tìm thấy món: {df.loc[dish_idx, 'name']}")
    
    # Lấy vector của món ăn này
    dish_vec = dish_vectors[dish_idx].reshape(1, -1)
    
    # Tính cosine similarity với tất cả các món ăn
    similarities = cosine_similarity(dish_vec, dish_vectors)[0]
    
    # Lấy top_n món ăn tương tự (bỏ qua chính nó)
    similar_indices = similarities.argsort()[::-1][1:top_n+1]
    
    # Tạo DataFrame kết quả
    recommendations = df.iloc[similar_indices][['name', 'type', 'Main Ingredients', 
                                                  'Calories (Approx.)', 'Suitable Time', 'score-text']].copy()
    recommendations['similarity_score'] = similarities[similar_indices]
    
    return recommendations

print("✅ Hàm recommend_dishes đã sẵn sàng!")

✅ Hàm recommend_dishes đã sẵn sàng!


In [14]:
# Test 1: Gợi ý cho món Phở
print("="*70)
print("TEST 1: GỢI Ý MÓN TƯƠNG TỰ PHỞ")
print("="*70)
result1 = recommend_dishes("Crispy Vietnamese Pancake", top_n=5)
if result1 is not None:
    display(result1)

print("\n\n")

# Test 2: Gợi ý cho món Bánh mì
print("="*70)
print("TEST 2: GỢI Ý MÓN TƯƠNG TỰ BÁNH MÌ")
print("="*70)
result2 = recommend_dishes("Banh mi", top_n=5)
if result2 is not None:
    display(result2)

print("\n\n")

# Test 3: Gợi ý cho món Bún chả
print("="*70)
print("TEST 3: GỢI Ý MÓN TƯƠNG TỰ BÚN CHẢ")
print("="*70)
result3 = recommend_dishes("Bun cha", top_n=5)
if result3 is not None:
    display(result3)

TEST 1: GỢI Ý MÓN TƯƠNG TỰ PHỞ
✅ Tìm thấy món: Crispy Vietnamese Pancake (Bánh xèo)


,name,type,Main Ingredients,Calories (Approx.),Suitable Time,score-text,similarity_score
54,Vietnamese Mini Savory Pancakes (Bánh khọt),Pancake,"Rice flour, coconut milk, shrimp, scallion oil",50 - 80 (per cake),Breakfast / Snack,4.3,0.917290
46,Hue Pancake (Bánh khoai),Pancake,"Rice flour, turmeric, shrimp, pork, egg (fried...",300 - 400,Snack / Main,4.3,0.902144
29,Steamed Tapioca Layer Cake (Bánh da lợn),Cake,"Tapioca starch, rice flour, mung bean, coconut...",150 - 250,Dessert / Snack,4.0,0.880975
111,Clam Rice (Cơm hến),Rice Dish,"Cold rice, baby clams, pork cracklings, shrimp...",300 - 450,Breakfast / Lunch,NaN,0.878773
127,Xôi vò,Rice Dish,"Glutinous rice, mung bean paste (loose grains)...",300 - 400,Breakfast / Festive,NaN,0.870363





TEST 2: GỢI Ý MÓN TƯƠNG TỰ BÁNH MÌ
✅ Tìm thấy món: Cold Cut Banh Mi (Bánh mì đặc biệt)


,name,type,Main Ingredients,Calories (Approx.),Suitable Time,score-text,similarity_score
12,Meat and Cold Cuts Bánh Mì (Bánh mì thịt),Sandwich,"Baguette, ham, head cheese, pate, pork floss, ...",450 - 550,Breakfast / Lunch,4.4,0.999985
192,Head Cheese Banh Mi (Bánh mì giò thủ),Sandwich,"Baguette, Vietnamese head cheese (ear/snout/mu...",400 - 500,Breakfast,NaN,0.966236
1,Bánh mì,Sandwich type,"Baguette, pate, cucumber, cilantro, pickled ca...",350 - 500,Breakfast / Lunch / Snack,4.6,0.941358
113,Grilled Chicken Bánh Mì (Bánh mì gà nướng),Sandwich,"Baguette, grilled chicken, cucumber, pickled c...",450 - 550,Breakfast / Lunch,4.0,0.935926
159,Pork Sausage Bánh Mì (Bánh mì chả lụa),Sandwich,"Baguette, Vietnamese pork roll (ch? l?a), cucu...",350 - 450,Breakfast,NaN,0.914018





TEST 3: GỢI Ý MÓN TƯƠNG TỰ BÚN CHẢ
❌ Không tìm thấy món ăn 'Bun cha'
